<a href="https://colab.research.google.com/github/nilanjan-coder/nilanjan-repo/blob/master/matching_edit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
class shop:

    
    def __init__(self,name,category,qlenmax,workslots):
        self.__name=name                                                                 #name of shop
        self.__category=category                                                         #category of shop based on items sold
        self.__qlencurr={}                                                               #current queue length in the shop
        self.__qlenmax=qlenmax                                                           #maximum queue length allowed in the shop
        self.__workslots=workslots                                                       #working slots of the shop
        self.__inventory={}                                                              #current inventory of the shop
        self.__prices={}                                                                 #prices of items in the shop
        self.__schedule={}                                                               #contains list of customers that have upcoming bookings in the working slots of the shop
        self.__inventoryforecast={{}}                                                    #expected inventory at the start of each working slot of the shop
        self.__revenue=0                                                                 #current revenue
        self.__revenueforecast=0                                                         #expected revenue of the day

    def get_inventory(self):
        return self.__inventory

    def get_inventory_forecast(self):
        return self.__inventoryforecast

    def get_queuelength(self):
        return self.__qlencurr

    def get_itemprice(self):
        return self.__prices

    def get_schedule(self):
        return self.__schedule

    def get_revenue(self):
        return self.__revenue

    def get_revenue_forecast(self):
        return self.__revenueforecast

    def update_queue_length(self,slot):
        self.__qlencurr[slot] = len(self.__schedule[slot])

    def update_price(self,item,price):
        self.__prices[item]=price

    def update_inventory_and_revenue(self,item,quantity,request):
        if(request=='credit'):
            if item not in self.__inventory.keys():
                self.__inventory[item] = 0
            self.__inventory[item] += quantity
        elif(request=='debit'):
            self.__inventory[item] -= quantity
            self.__revenue += self.__prices[item] * quantity
            if(self.__inventory[item]==0):
                print('need more'+item)

    def update_working_slots(self,add_slot,del_slot):
        if del_slot is not None:
            self.__workslots.remove(del_slot)
        if add_slot is not None:
            self.__workslots.append(add_slot)
        self.__workslots.sort()

    def update_schedule(self,slot,customer_name,add_or_del):
        if(add_or_del=='add'):
            self.__schedule[slot].append(customer_name)
        if(add_or_del=='del'):
            self.__schedule[slot].remove(customer_name)

    def update_inventory_forecast_and_revenue_forecast(self,slot,item,quantity,request):
        if(request=='credit'):
            if item not in self.__inventoryforecast[slot].keys():
                self.__inventoryforecast[slot][item] = 0
            self.__inventoryforecast[slot][item] += quantity
        elif(request=='debit'):
            self.__inventoryforecast[slot][item] -= quantity
        next_slot_index =  self.__workslots.index(slot)+1
        if(next_slot_index == len(self.__workslots)):
            self.__revenueforecast += self.__prices[item] * quantity
            if(self.__inventoryforecast[slot][item]==0):
                print('need more'+item)
            return
        update_inventory_forecast_and_revenue_forecast(self,self.__workslots[next_slot_index],item,quantity,request)

    def metrics(self,slot,item,quantity):
        fraction=0    
        total_cost=0     
        quant=0  
        qempty=0
        if(qlencurr[slot]<qlenmax):
            qempty = (qlenmax-qlencurr[slot])/qlenmax
            if item in self.__inventoryforecast[slot].keys():
                if(quantity>self.__inventoryforecast[self.__workslots[-1]][item]):
                    if(self.__inventoryforecast[self.__workslots[-1]][item]>0):
                        quant = self.__inventoryforecast[self.__workslots[-1]][item]
                        fraction = quant/quantity
                else:
                    quant = quantity
                    fraction = 1
                total_cost += self.__prices[item] * quant
            return round(fraction,2), qempty, total_cost, quant
    


In [0]:
class customer:

   
    def __init__(self,name):
        self.__name=name                            #name of customer
        self.__demand={}                            #names of items with quantities
        self.__slots=[]                             #slots available to go to market
        self.__shops={}                             #names of items to be bought in corresponding shops 
        self.__weightage={}                         #weights given to quantity_of_item_fulfiled, length_of_empty_queue, price_of_item, quick_delivery, ordering_of_shops

    def give_request(self,demand,slots,weightage):
        self.__demand = demand
        self.__slots = slots
        self.__slots.sort()
        self.__shops = shops
        self.__weightage=weightage

    def modify_demand(self,item,quantity):
        demand[item] = quantity

    def modify_timeslots(self,add_slot,del_slot):
        if add_slot is not None:
            self.__slots.append(add_slot)
        if del_slot is not None:
            self.__slots.remove(del_slot)
        self.__slots.sort()

    def modify_shops(self,item,add_shop,pos,del_shop):
        if del_shop is not None:
            self.__shops[item].remove(del_shop)
        if add_shop is not None:
            self.__shops[item].insert(pos,add_shop)

    def modify_weightage(self,quantitem,qempty,priceitem,fastdelivery):
        weightage[quantity]=quantitem
        weightage[queue]=qempty
        weightage[cost]=priceitem
        weightage[quick]=fastdelivery
        weightage[shop]=orderingofshops

    def see_demand(self):
        return self.__demand

    def see_timeslots(self):
        return self.__slots

    def see_shops(self):
        return self.__shops

    def see_weightage(self):
        return self.__weightage


In [0]:
class market:


    def __init__(self,name,shops,categories,maxlenq,customers):
        self.__name=name                                                        #name of market
        self.__shopss=shopss                                                    #names of categories and the corresponding shops which fall in that category
        self.__items=items                                                      #names of categories and the corresponding items which fall in that category
        self.__maxlenq=maxlenq                                                  #names of shops and the maximum queue length permitted for that shop 
        self.__workhrs=workhrs                                                  #names of shops and their corresponding working hours
        self.__customers=customers                                              #names of customers

    def process_request(self,customer_name):
        customer_curr = customer(customer_name)
        demand = customer_curr.see_demand()
        slots = customer_curr.see_timeslots()
        shops = customer_curr.see_shops()
        weightage = customer_curr.see_weightage()
        for item in demand.keys():
            result={}
            num_shops = len(shops[item])
            shop_wt = {}
            shops_list=[]
            cnt = 0
            for shop_name in shops[item]:
                shop_curr = shop(shop_name,category,self.__maxlenq[shop_name],self.__workhrs[shop_name])
                shop_wt.update({shop_name:(num_shops-cnt)/num_shops})
                cart = []
                queue = []
                cost = []
                fulfilable = []
                slots_list = []
                for slot in slots:
                    if slot in shop_curr.__workslots:
                        cart_metric,queue_metric,cost_metric,fulfilable_quantity = shop_curr.metrics(slot,item,demand[item])
                        if(cart_metric>0):
                            slots_list.append(slot)
                            cart.append(cart_metric)
                            queue.append(queue_metric)
                            cost.append(cost_metric)
                            fulfilable.append(fulfilable_quantity)
                num_slots = len(slots_list)
                slot_wt = [(num_slots-i)/num_slots for i in range(num_slots)]
                cost_norm=[(max(cost)-i)/max(cost) for i in cost]
                shop_score = weightage[quick]*slot_wt + weightage[quantity]*cart + weightage[queue]*queue + weightage[cost]*cost_norm
                sorted_slots = list[sorted(zip(shop_name,slots_list,queue,cost,fulfilable,shop_score), key = lambda y:y[5])]
                shops_list.append(sorted_slots[-1])
                cnt += 1
            sorted_shops = sorted(shops_list, key = lambda y: y[5]+shop_wt[y[0]]*weightage[shop])
            result[item]=sorted_shops[-1]
